<h3>Chapter 4: Linear Algebra</h3>

Linear algebra is the branch of mathematics that deals with <i>vector spaces</i>. It underpins a large number of data science concepts and techniques, and will be used in subsequent chapters.

Good refresher resource: http://cs229.stanford.edu/summer2020/cs229-linalg.pdf

<i>Vectors</i> are objects that can be added together (to form new vectors) and that can be multiplied by <i>scalars</i> (i.e., numbers) to form new vectors. Vectors are points in some finite-dimensional space. For example, if you have heights, weights, and ages of a large number of people, you can treat your data as three-dimensional vectors (height, weight, age). If you're teaching a class with four exams, you can treat student grades as four-dimensional vectors (exam1, exam2, exam3, exam4).


In [1]:
# The simplest, from-scratch approach is to represent vectors as lists of numbers. A list of
# three numbers corresponds to a vector in three-dimensional space, and vice versa:

height_weight_age = [70,  # inches
                     170, # pounds
                     40]  # years

# However, Python lists aren't vectors, so in order to perform arithmetic on them,
# we'll build these tools ourselves
# In reality, these will have terrible performance, so we'd actually use NumPy

def vector_add(v, w):
    """Vector addition is componentwise"""
    return [v_i + w_i for v_i, w_i in zip(v, w)]

vector_add([1, 2], [2, 7])

[3, 9]

In [2]:
# Subtraction is similar to addition

def vector_subtract(v, w):
    """Vector subtraction is componentwise"""
    return [v_i - w_i for v_i, w_i in zip(v, w)]

vector_subtract([1, 2], [2, 7])

[-1, -5]

In [3]:
def vector_sum(vectors):
    """Componentwise sum of a list of vectors"""
    result = vectors[0]
    for vector in vectors[1:]:
        result = vector_add(result, vector)
    return result

vector_sum([[1,2], [3,4], [5,6]])

[9, 12]

In [4]:
# We're really just 'reduce-ing' the list of vectors using vector_add, so let's use
# a higher order function to simplify
from functools import reduce

def vector_sum(vectors):
    """Simpler implementation for vector_sum"""
    return reduce(vector_add, vectors)
vector_sum([[1,2], [3,4], [5,6]])

[9, 12]

In [5]:
def scalar_multiply(n, v):
    """n is a number, v is a vector"""
    return [n * v_i for v_i in v]

scalar_multiply(5, [1, 2, 3])

[5, 10, 15]

In [6]:
def vector_mean(vectors):
    """compute the vector whose ith element is the mean of the
    ith element of the input vectors"""
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))

vector_mean([[1,2], [3,4], [5,6]])

[3.0, 4.0]

In [7]:
def dot(v, w):
    """The dot product of two vectors is the sum of their componentwise products
    v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

# 1*4 + 2*5 + 3*6 = 4 + 10 + 18 = 32
dot([1,2,3], [4,5,6])

# The dot product measures how far the vector v extends in the w direction.
# It's the length of the vector you'd get if you 'projected' v onto w

32

In [8]:
# This will be used to compute its 'magnitude' (or length)

def sum_of_squares(v):
    """v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

sum_of_squares([4,5,6])

77

In [9]:
import math

def magnitude(v):
    return math.sqrt(sum_of_squares(v))

We now have all the pieces we need to compute the distance between two vectors, defined as:

$\sqrt{(v_{1} - w_{1})^{2} + ... + (v_{n} - w_{n})^{2}}$

In [10]:
def squared_distance(v, w):
    """(v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2"""
    return sum_of_squares(vector_subtract(v, w))

In [11]:
def distance(v, w):
    return math.sqrt(squared_distance(v, w))

# Equivalently, and more simply:
def distance(v, w):
    return magnitude(vector_subtract(v, w))

In [12]:
# Note that in real Python code, we would use the NumPy library since it has high-performance arrays and
# arithmetic operations included. The implementations here would have terrible performance.

<h3>Matrices</h3>

A <i>matrix</i> is a two-dimensional collection of numbers. We will represent matrices as lists of lists, with each inner list having the same size and representing a <i>row</i> of the matrix. If A is a matrix, then $A[i][j]$ is the element in the <i>ith</i> row and <i>jth</i> column.

In [13]:
# A has 2 rows and 3 columns
A = [[1, 2, 3],
     [4, 5, 6]]

# B has 3 rows and 2 columns
B = [[1, 2],
     [3, 4],
     [5, 6]]

In [14]:
def shape(A):
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0
    return num_rows, num_cols
shape(A)

(2, 3)

In [15]:
def get_row(A, i):
    return A[i]

def get_column(A, j):
    return [A_i[j] for A_i in A]

In [16]:
print(get_row(A, 0))
print(get_column(A, 0))

[1, 2, 3]
[1, 4]


In [17]:
def make_matrix(num_rows, num_cols, entry_fn):
    """returns a num_rows x num_cols matrix whose
    (i, j) entry is entry_fn(i, j)"""
    return [[entry_fn(i, j)           # given i, create a list
            for j in range(num_cols)] # [entry_fn(i, 0), ... ]
            for i in range(num_rows)] # create one list for each i

In [18]:
def is_diagonal(i, j):
    """1's on the 'diagonal', 0's everywhere else"""
    return 1 if i == j else 0

identity_matrix = make_matrix(5, 5, is_diagonal)
identity_matrix

[[1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1]]

Matrices will be important for several reasons.

1. A matrix can be used to represent a data set consisting of multiple vectors, simply by considering each vector as a row of the matrix. For example, if you had the heights, weights, and ages of 1,000 people you could put them in a 1,000 x 3 matrix

2. An n x k matrix can represent a linear function that maps k-dimensional vectors to n-dimensional vectors. 

3. Matrices can be used to represent binary relationships. For example, a matrix representing friendships could be A[i][j] = 1 if nodes i and j are connected and 0 otherwise.

In [19]:
# In chapter 1, we had:

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
                   (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]


# This could be represented as a matrix:
     #     user 0  1  2  3  4  5  6  7  8  9
friendships = [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0], # user 0
               [1, 0, 1, 1, 0, 0, 0, 0, 0, 0], # user 1 
               [1, 1, 0, 1, 0, 0, 0, 0, 0, 0], # user 2 
               [0, 1, 1, 0, 1, 0, 0, 0, 0, 0], # user 3 
               [0, 0, 0, 1, 0, 1, 0, 0, 0, 0], # user 4 
               [0, 0, 0, 0, 1, 0, 1, 1, 0, 0], # user 5 
               [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], # user 6 
               [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], # user 7 
               [0, 0, 0, 0, 0, 0, 1, 1, 0, 1], # user 8 
               [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]] # user 9

In [20]:
friendships[0][2] == 1 # True, 0 and 2 are friends

True

In [21]:
friendships[0][8] == 1 # False, 0 and 8 are not friends

False

In [22]:
# Only need to look at single row to determine friends

friends_of_five = [i for i, is_friend in enumerate(friendships[5]) if is_friend]
friends_of_five

[4, 6, 7]